### Inflation

This notebook aims to correct budget and revenue values in our dataframe for inflation. 
We are using Headline consumer price index (CPI) inflation to correct our dataframes, which is the most widely used measure of inflation. Our final correction uses CPI of the United States to correct for all movie revenues/budgets. The reason for this is that all values are in US Dollars. 

Data from https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG

In [32]:
import pandas as pd
from inflation import correct_for_inflation

In [33]:
data_folder = './data/'

In [34]:
# load inflation data
inflation = pd.read_csv(data_folder + 'inflation.csv', header=2)

# we only need the data for United Stated (country code USA)
inflation = inflation.loc[inflation['Country Code']=='USA']

# only include inflation data from 2000 to 2012
inflation = inflation[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '2000', '2001', '2002', '2003', '2004',
                       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012']]

inflation

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
251,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,3.376857,2.826171,1.586032,2.270095,2.677237,3.392747,3.225944,2.852672,3.8391,-0.355546,1.640043,3.156842,2.069337


In [35]:
# create index multipliers for every year from 2000-2012
# prices will be indexed to 2000 prices
years = ['2000', '2001', '2002', '2003', '2004','2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012']
inflation['2000'] = 1
for i in range(len(years) - 1):
    inflation[years[i+1]] = inflation[years[i]] * (1 + inflation[years[i+1]] / 100)
    
inflation

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012
251,United States,USA,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,1,1.028262,1.04457,1.068283,1.096883,1.134098,1.170683,1.204079,1.250305,1.245859,1.266292,1.306267,1.333298


In [36]:
# Load movies
movies = pd.read_table(data_folder + 'movie.metadata.tsv', names=['wiki_id','freebase_id','name','release_date','box_office_revenue','length','language','country','genre'])
movies = movies[movies['box_office_revenue'].notna()]
movies = movies.loc[(movies['release_date'] >= '2000') & (movies['release_date'] <= '2012')]
# (movies['box_office_revenue'] / inflation.iloc[0]['2007'])

In [37]:
# Correcting movie revenue corresponding to movie year
for i in range(len(years) - 1):
    movies.loc[(movies['release_date'] >= years[i]) & (movies['release_date'] < years[i+1]), 'box_office_revenue'] /= inflation.iloc[0][years[i]]
    
movies

,wiki_id,freebase_id,name,release_date,box_office_revenue,length,language,country,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,1.362575e+07,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
29,156558,/m/014k4y,Baby Boy,2001-06-27,2.857410e+07,123.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
53,12008535,/m/02vlsqt,Rudo y Cursi,2008,8.871331e+06,103.0,"{""/m/06nm1"": ""Spanish Language""}","{""/m/0b90_r"": ""Mexico"", ""/m/09c7w0"": ""United S...","{""/m/01z02hx"": ""Sports"", ""/m/07s9rl0"": ""Drama""..."
60,5708633,/m/0f0d1p,The Great New Wonderful,2006-06-23,1.469697e+05,87.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/0hj3n0w"": ""Ensembl..."
119,3873713,/m/0b4f1s,Lee Dae-ro Can't Die,2005-08-18,4.176018e+06,108.0,"{""/m/02hwhyv"": ""Korean Language""}","{""/m/06qd3"": ""South Korea""}","{""/m/02kdv5l"": ""Action"", ""/m/01z4y"": ""Comedy""}"
...,...,...,...,...,...,...,...,...,...
81634,22427855,/m/05zkcsk,Adam,2009-01-20,2.046463e+06,99.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0219x_"": ""Indie"", ""/m/01t_vv"": ""Comedy-dr..."
81659,25920477,/m/0b6lqyd,Source Code,2011-03-11,1.127891e+08,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
81668,4492078,/m/0c5clj,How to Eat Fried Worms,2006-08-25,1.113924e+07,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hqxf"": ""Family Film"", ""/m/07s9rl0"": ""Dra..."
81725,1918494,/m/0660qx,State and Main,2000-08-26,6.944471e+06,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana..."


In [38]:
movies = pd.read_table(data_folder + 'movie.metadata.tsv', names=['wiki_id','freebase_id','name','release_date','box_office_revenue','length','language','country','genre'])

In [39]:
movies = movies[movies['box_office_revenue'].notna()]
movies

,wiki_id,freebase_id,name,release_date,box_office_revenue,length,language,country,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ..."
13,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa..."
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""..."
21,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
...,...,...,...,...,...,...,...,...,...
81695,54540,/m/0f7hw,Coming to America,1988-06-29,288752301.0,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/03p5xs"": ""..."
81720,7761830,/m/0kvgqb,Spaced Invaders,1990,15369573.0,100.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3mt0"": ""Alien Film"", ""/m/06n90"": ""Scie..."
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana..."
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."


In [40]:
correct_for_inflation(movies, 'box_office_revenue', 2000, 2012)

,wiki_id,freebase_id,name,release_date,box_office_revenue,length,language,country,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,1.362575e+07,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3.600000e+06,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ..."
13,171005,/m/016ywb,Henry V,1989-11-08,1.016110e+07,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa..."
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,1.022727e+08,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""..."
21,612710,/m/02wjqm,New Rose Hotel,1999-10-01,2.152100e+04,92.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
...,...,...,...,...,...,...,...,...,...
81695,54540,/m/0f7hw,Coming to America,1988-06-29,2.887523e+08,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/03p5xs"": ""..."
81720,7761830,/m/0kvgqb,Spaced Invaders,1990,1.536957e+07,100.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3mt0"": ""Alien Film"", ""/m/06n90"": ""Scie..."
81725,1918494,/m/0660qx,State and Main,2000-08-26,6.944471e+06,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana..."
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,2.288622e+07,107.0,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
